# Risk-free rates

For this exercise, I was not given the risk-free rate curves or yield curves, but they are required for some models, such as Heston. Here, I attempt to solve for the risk-free rate and yield via MSE minimization using the data I was given. In a real-life case, this would not be necessary - risk-free rate and yields can be found in other databases.

In [13]:
# Common imports
import time, os, math
import pandas as pd
import numpy as np
root_dir = '/mnt/c/Users/Steve/implied_vol_machine_learning'

## Import data

In [2]:
import os
import pandas as pd
import datetime as dt

raw_data = pd.read_csv(os.path.join(root_dir, 'options_20220824.csv'))
call_data = raw_data.loc[(raw_data["Type"] == "call") & (raw_data["Ask"] < 99000.0)].copy()
call_data.loc[:, "maturity"] = (pd.to_datetime(call_data["Expiration"]) - pd.to_datetime(call_data[" DataDate"])).dt.days / 365
call_data.loc[:, "moneyness"] = call_data["Strike"] / call_data["UnderlyingPrice"]
call_data.loc[:, "ticker"] = call_data["UnderlyingSymbol"]
call_data.loc[:, "Mid"] = (call_data["Bid"]+call_data["Ask"])/2

print(call_data.iloc[0])

UnderlyingSymbol                   A
UnderlyingPrice               133.67
Exchange                           *
OptionSymbol        A220916C00060000
Blank                            NaN
Type                            call
Expiration                09/16/2022
 DataDate           08/24/2022 16:00
Strike                          60.0
Last                             0.0
Bid                             72.5
Ask                             75.1
Volume                             0
OpenInterest                       0
IV                            0.3615
Delta                         0.9997
Gamma                            0.0
Theta                        -0.8933
Vega                             0.0
Alias               A220916C00060000
maturity                    0.060274
moneyness                   0.448867
ticker                             A
Mid                             73.8
Name: 0, dtype: object


In [3]:
focus_data = call_data.loc[(call_data["maturity"] > 0.0) & (call_data["moneyness"] <= 2.5) & (call_data["IV"] > 0.0) & (call_data["Mid"] > 0.0)].copy()

## Define pricing function

In [41]:
import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

def bs_call(S, X, T, r, y, sigma):
    b = r - y
    sig_sqrt_t = sigma * np.sqrt(T)
    d1 = (np.log(S/X) + (b+sigma*sigma/2)*T)/sig_sqrt_t
    d2 = d1 - sig_sqrt_t
    opt_val = S * np.exp(-y*T) * norm.cdf(d1) - X * np.exp(-r*T) * norm.cdf(d2)
    return opt_val

print(bs_call(100, 95, 0.5, 0.1, 0.05, 0.2))

def bscall_mse(model_params, spot_prices, strikes, T, implied_vols, opt_price):
    r, y = model_params
    val = np.array([bs_call(S, X, T, r, y, sigma) for S, X, sigma in zip(spot_prices, strikes, implied_vols)]) - opt_price
    return np.sqrt((val * val).sum())

def batch_solve_call(start_values, bounds, spot_prices, strikes, maturities, implied_vols, opt_prices):
    allres = []
    for idx, T in enumerate(maturities):
        start = time.time()
        res = minimize(bscall_mse, start_values, bounds=bounds, args=(spot_prices[idx], strikes[idx], T, implied_vols[idx], opt_prices[idx]), tol=1e-3, method="SLSQP")
        end = time.time()
        allres.append((end-start, res))
    return allres

9.628983522021265


## Solve for the risk-free rate and yield

In [59]:
import copy, concurrent, multiprocessing

r = 0.02; y = 0.0
start_values = [r, y]
bounds = [(-1.0, 1.0), (-1.0, 1.0)]
# For the first case, SLSQP ended up fastest and just about as accurate
# Hopefully, that is true for the others
# try_methods = ['Nelder-Mead', 'Powell', 'CG', 'BFGS', 'L-BFGS-B', 'TNC', 'COBYLA', 'SLSQP', 'trust-constr'] 

# Create a list of concurrent tasks and run together
do_threading = True
tasks = []
all_task_results = []
results = []
spot_prices = []
strikes = []
maturities = []
implied_vols = []
opt_prices = []
grouped_df = focus_data.groupby(['ticker', 'Expiration'])
cpus = multiprocessing.cpu_count()
batch_size = 100 # math.ceil(len(grouped_df) / cpus)
print(f'Total solves: {len(grouped_df)}, CPUs: {cpus}, Batch size: {batch_size}')

start = time.time()
with concurrent.futures.ProcessPoolExecutor() as executor:
    for idx, (key, data) in enumerate(grouped_df):
        results.append([key[0], key[1], data.shape[0]])
        spot_prices.append(data['UnderlyingPrice'].to_numpy())
        strikes.append(data['Strike'].to_numpy())
        maturities.append(data['maturity'].iloc[0])
        implied_vols.append(data['IV'].to_numpy())
        opt_prices.append(data['Mid'].to_numpy())
        if idx % batch_size == batch_size-1:
            if do_threading:
                tasks.append(executor.submit(batch_solve_call, start_values.copy(), bounds.copy(), spot_prices.copy(), strikes.copy(), maturities.copy(), implied_vols.copy(), opt_prices.copy()))
            else:
                batched_results = batch_solve_call(start_values, bounds, spot_prices.copy(), strikes.copy(), maturities.copy(), implied_vols.copy(), opt_prices.copy())
                for result in batched_results:
                    all_task_results.append(result)
            spot_prices = []
            strikes = []
            maturities = []
            implied_vols = []
            opt_prices = []
        #if idx + 1 >= batch_size * cpus:
            #break
    if len(spot_prices) > 0:
        if do_threading:
            tasks.append(executor.submit(batch_solve_call, start_values, bounds, spot_prices.copy(), strikes.copy(), maturities.copy(), implied_vols.copy(), opt_prices.copy()))
        else:
            batched_results = batch_solve_call(start_values, bounds, spot_prices.copy(), strikes.copy(), maturities.copy(), implied_vols.copy(), opt_prices.copy())
            for result in batched_results:
                all_task_results.append(result)
    concurrent.futures.wait(tasks)    
end = time.time()

if do_threading:
    for task in tasks:
        for result in task.result():
            all_task_results.append(result)

# Gather results
# Each element is a list of [ticker, Expiration, count, time, risk_free_rate, yield_rate]
errors = []
for idx, (result, task_res) in enumerate(zip(results, all_task_results)):
    time_diff, res = task_res
    if res.success:
        result.append(time_diff)
        result.append(res.x[0])
        result.append(res.x[1])
    else:
        errors.append(result)
        del results[idx]
print('Sample values: ', results[0:3])
print('Problem cases: ', errors)
print(f'Total solve time: {end-start}s')

Total solves: 30449, CPUs: 8, Batch size: 100
Sample values:  [['A', '01/19/2024', 35, 0.16676592826843262, 0.02022078300872528, -0.00022528878441082558], ['A', '01/20/2023', 40, 0.599407434463501, 0.004701275696735967, -0.012945813410691473], ['A', '02/17/2023', 29, 0.15398073196411133, 0.02040405787846011, -0.00041230395759763805]]
Problem cases:  []
Total solve time: 916.9154798984528s


In [60]:
# Sanity check: see that we reached the end
print(results[-1])

['ZYXI', '11/18/2022', 7, 0.07892608642578125, 0.7233937287897988, 0.5452781853017455]


In [61]:
r_and_y_df = pd.DataFrame(results).rename(columns={0: 'ticker', 1: 'Expiration', 2: 'Count', 3: 'SolveTime', 4: 'RiskFreeRate', 5: 'YieldRate'})
print(r_and_y_df.iloc[0:3])
print('Solve times (sum, mean, median, max):', r_and_y_df['SolveTime'].sum(), r_and_y_df['SolveTime'].mean(), r_and_y_df['SolveTime'].median(), r_and_y_df['SolveTime'].max())
print('Number of solves over a second:', r_and_y_df[r_and_y_df['SolveTime'] > 1.0].shape[0])

  ticker  Expiration  Count  SolveTime  RiskFreeRate  YieldRate
0      A  01/19/2024     35   0.166766      0.020221  -0.000225
1      A  01/20/2023     40   0.599407      0.004701  -0.012946
2      A  02/17/2023     29   0.153981      0.020404  -0.000412
Solve times (sum, mean, median, max): 7142.086890697479 0.23455899670588456 0.1456770896911621 8.097209215164185
Number of solves over a second: 627


In [62]:
# This method of getting the risk-free rate can have surprisingly large variance
# For now, treat each ticker as having its own risk-free rate
r_and_y_df_clean = r_and_y_df[(r_and_y_df['RiskFreeRate'] > -0.9) & (r_and_y_df['RiskFreeRate'] < 0.9)]
rfr_analysis = []
for expiration, clean_data in r_and_y_df_clean.groupby('Expiration'):
    rfr_analysis.append([expiration, clean_data['RiskFreeRate'].mean(), clean_data['RiskFreeRate'].median(), clean_data['RiskFreeRate'].min(), clean_data['RiskFreeRate'].max(), clean_data['RiskFreeRate'].std()])
rfr_analysis_df = pd.DataFrame(rfr_analysis).rename(columns={0:'Expiration', 1:'Mean', 2:'Median', 3:'Min', 4:'Max', 5:'Std'})
print(rfr_analysis_df.to_string())

    Expiration      Mean    Median       Min       Max       Std
0   01/18/2023  0.033070  0.033070  0.027540  0.038599  0.007820
1   01/19/2024  0.035220  0.025249 -0.736571  0.861220  0.113843
2   01/20/2023  0.017578  0.018980 -0.840695  0.880304  0.129794
3   01/31/2023  0.031817  0.027694  0.026633  0.041124  0.008078
4   02/15/2023  0.031838  0.031838  0.027931  0.035746  0.005526
5   02/17/2023  0.020541  0.021339 -0.884499  0.890826  0.127459
6   03/15/2024 -0.008683  0.022709 -0.105418  0.025269  0.064514
7   03/17/2023  0.024229  0.022300 -0.719393  0.874374  0.100926
8   03/22/2023  0.027464  0.027464  0.027464  0.027464       NaN
9   03/31/2023  0.028737  0.023351 -0.298992  0.227095  0.099760
10  04/19/2023  0.027910  0.027910  0.027910  0.027910       NaN
11  04/21/2023  0.037734  0.024586 -0.857529  0.890061  0.109142
12  05/19/2023  0.022892  0.021951 -0.030932  0.104744  0.025132
13  06/16/2023  0.026739  0.023979 -0.625252  0.516268  0.063145
14  06/21/2024  0.032038 

In [63]:
# Save for use in other notebooks
r_and_y_df.to_csv(os.path.join(root_dir, 'riskfreerate_and_yield.csv'))

print('Done')

Done
